## 1. Farmers' Market Expansion
<p>You have been hired by a farming organisation that helps local farmers sell their products. They want to know whether they should open up a new farmers' market to sell dairy products from nearby farmers. They have supplied you with daily shopping data from a panel of local households from 2019 to 2020. </p>
<p>The organization will make their decision based on whether dairy products are popular in the area, and whether sales are trending in a positive direction. To answer these questions, they want three pieces of data:</p>
<ol>
<li>What was the total number of purchases of dairy products for each month of 2020 (i.e., the <code>total_sales</code>)?</li>
<li>What was the total share of dairy products (out of all products purchased) for each month of 2020 (i.e., the <code>market_share</code>)?</li>
<li>For each month of 2020, what was the percentage increase or decrease in total monthly dairy purchases compared to the same month in 2019 (i.e., the <code>year_change</code>)?</li>
</ol>
<p>The organization handles not only dairy farmers, but also those with chicken farms. As a result, they are only interested in these three categories (which they treat as dairy): ‘whole milk’, 'yogurt' and 'domestic eggs'.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5960/datasets/463543c8c38957ca5b95d93b02f2cb1bec53334f/diagram.PNG" alt="Database Schema" width="400px"></p>

In [24]:
%%sql
postgresql:///groceries

CREATE TABLE total_sales AS
SELECT 
    SUM(sub.dairy) as total_sales, 
    ROUND(sub.month) as month
FROM
    (SELECT EXTRACT(MONTH FROM p20.fulldate) as month, cat.category,
    CASE WHEN cat.category = 'whole milk' 
        OR cat.category = 'yogurt'
        OR cat.category = 'domestic eggs' THEN 1 ELSE NULL END AS dairy
    FROM purchases_2020 AS p20
    JOIN categories AS cat
    ON p20.purchaseid = cat.purchase_id) as sub
GROUP BY month
ORDER BY month;

(psycopg2.errors.DuplicateTable) relation "total_sales" already exists

[SQL: CREATE TABLE total_sales AS
SELECT 
    SUM(sub.dairy) as total_sales, 
    ROUND(sub.month) as month
FROM
    (SELECT EXTRACT(MONTH FROM p20.fulldate) as month, cat.category,
    CASE WHEN cat.category = 'whole milk' 
        OR cat.category = 'yogurt'
        OR cat.category = 'domestic eggs' THEN 1 ELSE NULL END AS dairy
    FROM purchases_2020 AS p20
    JOIN categories AS cat
    ON p20.purchaseid = cat.purchase_id) as sub
GROUP BY month
ORDER BY month;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [25]:
%%sql
postgresql:///groceries
    
ALTER TABLE purchases_2020 ALTER COLUMN fulldate TYPE DATE 
using to_date(fulldate, 'YYYY-MM-DD'); 

(psycopg2.errors.UndefinedFunction) function to_date(date, unknown) does not exist
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.

[SQL: ALTER TABLE purchases_2020 ALTER COLUMN fulldate TYPE DATE 
using to_date(fulldate, 'YYYY-MM-DD');]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [26]:
%%sql
postgresql:///groceries

CREATE TABLE p20 AS
SELECT 
    SUM(sub.dairy) as dairy,
    COUNT(sub.dairy) as total,
    ROUND(sub.month) as month
FROM
    (SELECT EXTRACT(MONTH FROM p20.fulldate) as month, cat.category,
    CASE WHEN cat.category = 'whole milk' 
        OR cat.category = 'yogurt'
        OR cat.category = 'domestic eggs' THEN 1 ELSE 0 END AS dairy
    FROM purchases_2020 AS p20
    JOIN categories AS cat
    ON p20.purchaseid = cat.purchase_id) as sub
GROUP BY month
ORDER BY month;

(psycopg2.errors.DuplicateTable) relation "p20" already exists

[SQL: CREATE TABLE p20 AS
SELECT 
    SUM(sub.dairy) as dairy,
    COUNT(sub.dairy) as total,
    ROUND(sub.month) as month
FROM
    (SELECT EXTRACT(MONTH FROM p20.fulldate) as month, cat.category,
    CASE WHEN cat.category = 'whole milk' 
        OR cat.category = 'yogurt'
        OR cat.category = 'domestic eggs' THEN 1 ELSE 0 END AS dairy
    FROM purchases_2020 AS p20
    JOIN categories AS cat
    ON p20.purchaseid = cat.purchase_id) as sub
GROUP BY month
ORDER BY month;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [27]:
%%sql
postgresql:///groceries

CREATE TABLE market_share AS
SELECT *, 
ROUND((ROUND(dairy,2) * 100 )/ROUND(total,2),2) AS market_share
FROM p20;

(psycopg2.errors.DuplicateTable) relation "market_share" already exists

[SQL: CREATE TABLE market_share AS
SELECT *, 
ROUND((ROUND(dairy,2) * 100 )/ROUND(total,2),2) AS market_share
FROM p20;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [28]:
%%sql
postgresql:///groceries
    
select COLUMN_NAME,DATA_TYPE
from INFORMATION_SCHEMA."columns" 
where TABLE_NAME = 'p20'

3 rows affected.


column_name,data_type
dairy,bigint
total,bigint
month,double precision


In [29]:
%%sql
postgresql:///groceries

CREATE TABLE p19 AS
SELECT 
    SUM(sub.dairy) as total_sales, 
    ROUND(sub.month) as month
FROM
    (SELECT p19.month, cat.category,
    CASE WHEN cat.category = 'whole milk' 
        OR cat.category = 'yogurt'
        OR cat.category = 'domestic eggs' THEN 1 ELSE NULL END AS dairy
    FROM purchases_2019 AS p19
    JOIN categories AS cat
    ON p19.purchase_id = cat.purchase_id) as sub
GROUP BY month
ORDER BY month;

(psycopg2.errors.DuplicateTable) relation "p19" already exists

[SQL: CREATE TABLE p19 AS
SELECT 
    SUM(sub.dairy) as total_sales, 
    ROUND(sub.month) as month
FROM
    (SELECT p19.month, cat.category,
    CASE WHEN cat.category = 'whole milk' 
        OR cat.category = 'yogurt'
        OR cat.category = 'domestic eggs' THEN 1 ELSE NULL END AS dairy
    FROM purchases_2019 AS p19
    JOIN categories AS cat
    ON p19.purchase_id = cat.purchase_id) as sub
GROUP BY month
ORDER BY month;]
(Background on this error at: http://sqlalche.me/e/14/f405)


In [30]:
%%sql
postgresql:///groceries

SELECT 
    month,
    dairy as total_sales, 
    ROUND((ROUND(dairy,2) * 100 )/ROUND(total,2),2) AS market_share,
    ROUND((ROUND(total_sales,2) * 100 )/ROUND(dairy,2),2) AS year_change
FROM
    (SELECT dairy, total, CAST(p20.month as bigint), total_sales
    FROM p20
    JOIN p19
    ON p20.month = p19.month) as sub;

12 rows affected.


month,total_sales,market_share,year_change
1,228,12.47,67.54
2,165,11.11,95.15
3,216,12.05,78.24
4,218,13.09,79.36
5,195,11.40,85.13
6,196,10.94,82.65
7,195,11.85,100.51
8,231,11.77,75.76
9,180,11.16,95.56
10,193,11.61,84.97


In [31]:
%%sql
postgresql:///groceries

SELECT dairy, total, CAST(p20.month as bigint), total_sales
    FROM p20
    JOIN p19
    ON p20.month = p19.month;

12 rows affected.


dairy,total,month,total_sales
228,1829,1,154
165,1485,2,157
216,1792,3,169
218,1666,4,173
195,1710,5,166
196,1791,6,162
195,1645,7,196
231,1963,8,175
180,1613,9,172
193,1663,10,164


In [32]:
%%sql
postgresql:///groceries

SELECT *
FROM answer;

12 rows affected.


month,total_sales,market_share,year_change
1,228,12.47,67.54
2,165,11.11,95.15
3,216,12.05,78.24
4,218,13.09,79.36
5,195,11.40,85.13
6,196,10.94,82.65
7,195,11.85,100.51
8,231,11.77,75.76
9,180,11.16,95.56
10,193,11.61,84.97
